In [ ]:
import os
from groq import Groq
import pandas as pd
import numpy as np

In [ ]:
apiKey = "fill_it_here"

In [ ]:
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY"),
    api_key= apiKey,
)

In [ ]:
dataset = pd.read_csv("keyphrases/done/keyphrases_SpotifyMixtral_Done.csv")

In [ ]:
dataset

In [ ]:
score_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(score_counts_balanced)

In [ ]:
train_data = dataset[['content', 'sentiment', 'Merged keyphrases_reduced 2']]
train_data['row_number'] = train_data.index

In [ ]:
train_data

In [ ]:
list = []
i = 0
for index, row in train_data.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:


                1. Comment: "Terrible. I won't repeat what the other reviews have said, the continuous ads are so intrusive and time-consuming with the free version. Now that they are joining the likes of other silicone valley politics and starting to censor people, I am officially done with Spotify. I was honestly considering premium to avoid the ads, but then the censorship started."
                Topics / Keyphrases: ['Spotify ads frequency increase', 'User experience with excessive ads', 'Pressure to purchase Spotify Premium', 'Comparison of ads frequency with JioSaavn', 'Ads mid-song and poor timing', 'Ads from liberal companies and censorship', 'Song search and free version limitations', 'Users compelled to premium by irritating ads', 'Alternative music apps 0with less ads', 'Improve user experience by reducing ads']
                Label: 1

                2. Comment: "ads and limited skips are annoying. you could literally download songs from free music apps and listen to it offline, unlimited skips and choose whatever songs you want. i used Spotify for a few weeks now and i think I'll go back using Music instead of this app."
                Topics / Keyphrases: ['Spotify app issues', 'song identification problems', 'music playback interruptions', 'user experience with ads', 'limited skipping in free version', 'paying for premium features', 'annoying frequency of ads', 'user frustration with restrictions', 'comparison with YouTube music', 'recommendations for improvements']
                Label: 1

                3. Comment: "Poor music site with bad ethics. They were better in the past but the direction they went over the past 4 years is just a bad user experience. There are better music app services. Don't recommend at all."
                Topics / Keyphrases: ['user experience with Spotify app', 'issue with ads during music streaming', 'negative feedback on commercials in podcasts', 'premium subscription vs. free version', 'user complaints on company values', 'limited song selection on Spotify', 'program evaluation and user opinion', 'limited interruptions as user preference', 'fairness in ad system', 'overall positive reviews with concerns']
                Label: 1

                4. Comment: "Really glitchy lately. Just keeps stopping every 5 seconds. Then starts playing even when the app isnt open. I pay a subscription fee but am going to have to cancel. This is ridiculous now."
                Topics / Keyphrases: ['Spotify app issues', 'unexpected app behavior', 'playback errors', 'subscription fee', 'app crashes', 'startup problems', 'user experience issues', 'media control problems', 'disappearing playbars and liked songs', 'developer practices critique']
                Label: 1

                5. Comment: "Cannot login to my account at all. The app is repeatedly crashing. It sends the login link to my email, says it's valid for 4 hours but the link says it's expired. Nonsense."
                Topics / Keyphrases: ['Spotify app issues', 'Login problems', 'Uninstallation difficulties', 'Download stuck', 'Offline functionality', 'Connection errors', 'User experience problems', 'Internal storage permission', 'File access issues', 'Playback interruptions']
                Label: 1

                6. Comment: "I like the app but I wanted to mostly use it for offline purposes but ever since a few days after I downloaded my download audio kept dissapearing"
                Topics / Keyphrases: ['Spotify app issues', 'user experience problems', 'disappearing downloads', 'music playlist management', 'song availability', 'app freezing', 'censorship of podcasts', 'feature removal', 'unintended playlist changes', 'Spotify updates with Google Maps']
                Label: 2

                7. Comment: "Where is my Car Mode?! You deleted it? Whyy, it was so useful, now i need to lock into phone during driving to change song.. thats stupid."
                Topics / Keyphrases: ['user comment', 'Google Play Store', 'Spotify App', 'user experience', 'missing feature', 'phone use while driving', 'music playback', 'app crashes', 'last update problem', 'request for fix']
                Label: 2

                8. Comment: "GIVE THE OPTION TO DISABLE THE VOLUME SYNC WITH SPOTIFY CONNECT FOR CHRIST SAKE!!! I have multiple Connect devices and I don't want them locked together at the same volume. THIS IS A BIG ISSUE THAT IS AN EASY FIX FOR YOU!! Listen to your (paying) users because a quick Google search shows this is a common issue. 5 stars if this petty issue is fixed."
                Topics / Keyphrases: ['user control over content visibility', 'Spotify Connect volume sync issue', 'going for a drive notification complaint', 'Spotify app music playback stops', 'media bar change issues in update', 'Samsung S22 music streaming issues', 'shuffle default playlist concern', 'missing play button issue', 'pause unpause button placement complaint', 'Spotify update clear app issue']
                Label: 2

                9. Comment: "It used to be so good, I had premium for years and i still would. But its not worth it anymore. Its super glitchy and buggy, even in premium. They tried to simplify it and 'have better used experience' but all that they did was make it bad. Please go back a version or two and keep it that way. I used to love the app"
                Topics / Keyphrases: ['Spotify Premium issues', 'app glitches and bugs', 'poor audio quality', 'user experience decline', 'volume control problems', 'offline playback issues', 'feature removal', 'user complaints', 'alternative music services', 'Spotify app control problems']
                Label: 2

                10. Comment: "GREAT!app just don't like the so many apps and the constant add asking you to get Spotify premium. The other thing that annoys me is the limited skips and when I search up a song it won't let me play it it plays a playlist instead and I can't skip it all the time because I don't have anymore skips!"
                Topics / Keyphrases: ['Spotify app issues', 'user experience complaints', 'limited song skips', 'annoying ads', 'songs playlist mix ups', 'app instability', 'inaccurate podcast info', 'playlist starting automatically', 'song liking problem', 'user feedback']
                Label: 2

                11. Comment: "The free accounts are ok. I get the ads but I really dont think it is great that you cannot choose what song you want to play with a free account. It is a good music streaming service though and I will recommend but they need to work on some of the free account features."
                Topics / Keyphrases: ['Spotify app restrictions', 'premium subscription', 'free accounts', 'user experience', 'music streaming service', 'song limitation', 'additional cost', 'app functionality', 'offline playback issue', 'playlist navigation']
                Label: 3

                12. Comment: "I've strong network connection but sometimes this app shows there is a network problem so that lyrics can't be load. Don't know why does it happens! And after 2 or 3 Times reboot, the app shows the lyrics, you know when you're looking for lyrics and lyrics aren't here, it ruins the mood"
                Topics / Keyphrases: ['Spotify app issues', 'network connection problems', 'song playback issues', 'user experience', 'media bar disappears', 'song repetition', 'liked music not playing', 'payment issues', 'party connectivity', 'confusing interface']
                Label: 3

                13. Comment: "It has alot of good songs but there is the point that only premeum can skip ads and have free skips and they add songs of their own that they think you'll like but i hate them its a great app apart from that"
                Topics / Keyphrases: ['Spotify app', 'user experience', 'limited free skips', 'premium feature for ads skipping', 'songs selection', 'offensive ads', 'user complaints', 'deceitful programming', 'forced shuffle feature', 'music streaming']
                Label: 3

                14. Comment: "The app works well and I don't mind the ads. But I think that it should have an option to play in shuffle or play in the order the songs are in; I make my playlists in the order I like them, so shuffle being the only option makes it frustrating."
                Topics / Keyphrases: ['music streaming experience', 'Spotify app features', 'song customization', 'user frustration', 'playlist creation', 'song previews', '10k song limit', 'shuffle mode', 'period pad ads', 'Spotify Premium recommendation']
                Label: 3

                15. Comment: "I've loved the app so far, the interface... organization..but I was logged out this evening and it's refusing to log me in. This is sad. Please fix it"
                Topics / Keyphrases: ['Spotify app login issues', 'user experience problems', 'email password recovery', 'local files playback', 'account compatibility', 'M-Pesa payment issue', 'phone technical difficulties', 'internet access settings', 'data system recognition', 'app functionality fix']
                Label: 3

                16. Comment: "Great app, great music, love the weekly custom play lists but only negative thing is the pop aps if it not a primum user"
                Topics / Keyphrases: ['Spotify App', 'user comments', 'music streaming', 'positive opinion', 'flaw', 'song downloading', 'user experience', 'song selection', 'Google Play Store', 'app analysis']
                Label: 4

                17. Comment: "I love Spotify but it gets annoying when I try to add an album to a playlist, I have to go through every song and add them by hand. Other than that I find it wonderful 😊"
                Topics / Keyphrases: ['user experience', 'Spotify app', 'playlist customization', 'song selection', 'playback issues', 'premium membership', 'song order', 'music recommendations', 'technical problems', 'app functionality']
                Label: 4

                18. Comment: "I may not remember the song's name so it would be good to have a function where we can just say some key words and the artist. But is amazing."
                Topics / Keyphrases: ['Spotify app user experience', 'song identification and playback issues', 'offline listening limitations', 'song lyrics and information', 'playlist creation and customization', 'music preference and recommendations', 'app navigation and control', 'free version limitations', 'artist compensation and payment', 'user feedback and suggestions']
                Label: 4

                19. Comment: "Love it! Great among quality, perfect sound for songs. My one problem is when it adds songs to your playlists. Other than that its perfect."
                Topics / Keyphrases: ['Spotify app user experience', 'high quality music', 'great sound', 'songs and playlists', 'album cover concern', 'user interface issues', 'app evaluation', 'music variety', 'free version limitations', 'app improvement feedback']
                Label: 4

                20. Comment: "This is a good music app overall but the only thing anoying is there's so much ads that say thank you for listening to Spotify it's so anoying it's like made.ti anoy people but overall best music app ever"
                Topics / Keyphrases: ['Spotify App', 'music service', 'podcasts', '5 stars', 'user experience', 'music discovery', 'Joe Rogan Podcast', 'Google Play Store', 'fast forward', 'self-remove artists']
                Label: 4

                21. Comment: "SPOTIFY IS MY DAMN JAM!!! I'D PICK IT OVER ANY OTHER MUSIC APP OUT THERE ESPECIALLY NOW THAT THEY HAVE ADDED THE LYRICS TO SONGS!!! My only request is to have the option to keep ur smartphone screen awake if u have the lyrics pulled up!! Love love love you Spotify Folk!!!!"
                Topics / Keyphrases: ['Spotify app issues', 'trending albums refresh', 'annoying bugs', 'saving albums/playlists', 'user experience', 'music playback problems', 'app improvement requests', 'playlist limitations', 'hidden favorite tracks', 'app customization options']
                Label: 5

                22. Comment: "The app is great i regualry listen to lots of fantastic music and premium is very good because there are no adds! So i definetley reccomend you install it!:):):):)"
                Topics / Keyphrases: ['Spotify App', 'user experience', 'music streaming', 'playlist', 'song selection', 'premium feature', 'app performance', 'user feedback', 'WiFi connection issue', 'AI and 
                ads']
                Label: 5

                23. Comment: "I have this app for 8 years and counting and I cannot live without it! Spotify accompanies me everywhere I go. Thank you!"
                Topics / Keyphrases: ['best music streaming app', 'personalized music playlists', 'user experience', 'Spotify app usage', 'music preference', 'positive user feedback', 'app loyalty', 'Google Play Store', 'daily music listening', 'music app enhancement']
                Label: 5

                24. Comment: "so far this is the best music app I have used rarely crash's Plays offline has a multitude of features and functions and very easy to manuver around. Thanks so much guy's and gal's for all your hard work"
                Topics / Keyphrases: ['Spotify app usability', 'music customization and recommendations', 'app continuity and features', 'phone usage and music streaming', 'user-friendly interactive experience', 'satisfied long-term user experience', 'convenient music listening and accessibility', 'easy-to-navigate international platform', 'music discovery and song education', 'cost-effective artist platform']
                Label: 5

                25. Comment: "I just Love Spotify. It has every single song/music that is, was, and will be a part of my life's music repritoire. Thanks for supplying such a great array of listening pleasures, SPOTIFY!!"
                Topics / Keyphrases: ['Spotify as top music streaming platform', 'growing music and podcast options', 'supporting underground artists', 'Spotify for Artists', 'user preferences in audio entertainment', 'headphones usage', 'user experience', 'Spotify app features', 'user's love for Spotify Premium', 'playlist creation and song recommendations']
                Label: 5


                Please provide the sentiment classification as a single number (1, 2, 3, 4, or 5) and nothing more for the following comment:


                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
rest = train_data[len(list):]
for index, row in rest.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:


                1. Comment: "Terrible. I won't repeat what the other reviews have said, the continuous ads are so intrusive and time-consuming with the free version. Now that they are joining the likes of other silicone valley politics and starting to censor people, I am officially done with Spotify. I was honestly considering premium to avoid the ads, but then the censorship started."
                Topics / Keyphrases: ['Spotify ads frequency increase', 'User experience with excessive ads', 'Pressure to purchase Spotify Premium', 'Comparison of ads frequency with JioSaavn', 'Ads mid-song and poor timing', 'Ads from liberal companies and censorship', 'Song search and free version limitations', 'Users compelled to premium by irritating ads', 'Alternative music apps 0with less ads', 'Improve user experience by reducing ads']
                Label: 1

                2. Comment: "ads and limited skips are annoying. you could literally download songs from free music apps and listen to it offline, unlimited skips and choose whatever songs you want. i used Spotify for a few weeks now and i think I'll go back using Music instead of this app."
                Topics / Keyphrases: ['Spotify app issues', 'song identification problems', 'music playback interruptions', 'user experience with ads', 'limited skipping in free version', 'paying for premium features', 'annoying frequency of ads', 'user frustration with restrictions', 'comparison with YouTube music', 'recommendations for improvements']
                Label: 1

                3. Comment: "Poor music site with bad ethics. They were better in the past but the direction they went over the past 4 years is just a bad user experience. There are better music app services. Don't recommend at all."
                Topics / Keyphrases: ['user experience with Spotify app', 'issue with ads during music streaming', 'negative feedback on commercials in podcasts', 'premium subscription vs. free version', 'user complaints on company values', 'limited song selection on Spotify', 'program evaluation and user opinion', 'limited interruptions as user preference', 'fairness in ad system', 'overall positive reviews with concerns']
                Label: 1

                4. Comment: "Really glitchy lately. Just keeps stopping every 5 seconds. Then starts playing even when the app isnt open. I pay a subscription fee but am going to have to cancel. This is ridiculous now."
                Topics / Keyphrases: ['Spotify app issues', 'unexpected app behavior', 'playback errors', 'subscription fee', 'app crashes', 'startup problems', 'user experience issues', 'media control problems', 'disappearing playbars and liked songs', 'developer practices critique']
                Label: 1

                5. Comment: "Cannot login to my account at all. The app is repeatedly crashing. It sends the login link to my email, says it's valid for 4 hours but the link says it's expired. Nonsense."
                Topics / Keyphrases: ['Spotify app issues', 'Login problems', 'Uninstallation difficulties', 'Download stuck', 'Offline functionality', 'Connection errors', 'User experience problems', 'Internal storage permission', 'File access issues', 'Playback interruptions']
                Label: 1

                6. Comment: "I like the app but I wanted to mostly use it for offline purposes but ever since a few days after I downloaded my download audio kept dissapearing"
                Topics / Keyphrases: ['Spotify app issues', 'user experience problems', 'disappearing downloads', 'music playlist management', 'song availability', 'app freezing', 'censorship of podcasts', 'feature removal', 'unintended playlist changes', 'Spotify updates with Google Maps']
                Label: 2

                7. Comment: "Where is my Car Mode?! You deleted it? Whyy, it was so useful, now i need to lock into phone during driving to change song.. thats stupid."
                Topics / Keyphrases: ['user comment', 'Google Play Store', 'Spotify App', 'user experience', 'missing feature', 'phone use while driving', 'music playback', 'app crashes', 'last update problem', 'request for fix']
                Label: 2

                8. Comment: "GIVE THE OPTION TO DISABLE THE VOLUME SYNC WITH SPOTIFY CONNECT FOR CHRIST SAKE!!! I have multiple Connect devices and I don't want them locked together at the same volume. THIS IS A BIG ISSUE THAT IS AN EASY FIX FOR YOU!! Listen to your (paying) users because a quick Google search shows this is a common issue. 5 stars if this petty issue is fixed."
                Topics / Keyphrases: ['user control over content visibility', 'Spotify Connect volume sync issue', 'going for a drive notification complaint', 'Spotify app music playback stops', 'media bar change issues in update', 'Samsung S22 music streaming issues', 'shuffle default playlist concern', 'missing play button issue', 'pause unpause button placement complaint', 'Spotify update clear app issue']
                Label: 2

                9. Comment: "It used to be so good, I had premium for years and i still would. But its not worth it anymore. Its super glitchy and buggy, even in premium. They tried to simplify it and 'have better used experience' but all that they did was make it bad. Please go back a version or two and keep it that way. I used to love the app"
                Topics / Keyphrases: ['Spotify Premium issues', 'app glitches and bugs', 'poor audio quality', 'user experience decline', 'volume control problems', 'offline playback issues', 'feature removal', 'user complaints', 'alternative music services', 'Spotify app control problems']
                Label: 2

                10. Comment: "GREAT!app just don't like the so many apps and the constant add asking you to get Spotify premium. The other thing that annoys me is the limited skips and when I search up a song it won't let me play it it plays a playlist instead and I can't skip it all the time because I don't have anymore skips!"
                Topics / Keyphrases: ['Spotify app issues', 'user experience complaints', 'limited song skips', 'annoying ads', 'songs playlist mix ups', 'app instability', 'inaccurate podcast info', 'playlist starting automatically', 'song liking problem', 'user feedback']
                Label: 2

                11. Comment: "The free accounts are ok. I get the ads but I really dont think it is great that you cannot choose what song you want to play with a free account. It is a good music streaming service though and I will recommend but they need to work on some of the free account features."
                Topics / Keyphrases: ['Spotify app restrictions', 'premium subscription', 'free accounts', 'user experience', 'music streaming service', 'song limitation', 'additional cost', 'app functionality', 'offline playback issue', 'playlist navigation']
                Label: 3

                12. Comment: "I've strong network connection but sometimes this app shows there is a network problem so that lyrics can't be load. Don't know why does it happens! And after 2 or 3 Times reboot, the app shows the lyrics, you know when you're looking for lyrics and lyrics aren't here, it ruins the mood"
                Topics / Keyphrases: ['Spotify app issues', 'network connection problems', 'song playback issues', 'user experience', 'media bar disappears', 'song repetition', 'liked music not playing', 'payment issues', 'party connectivity', 'confusing interface']
                Label: 3

                13. Comment: "It has alot of good songs but there is the point that only premeum can skip ads and have free skips and they add songs of their own that they think you'll like but i hate them its a great app apart from that"
                Topics / Keyphrases: ['Spotify app', 'user experience', 'limited free skips', 'premium feature for ads skipping', 'songs selection', 'offensive ads', 'user complaints', 'deceitful programming', 'forced shuffle feature', 'music streaming']
                Label: 3

                14. Comment: "The app works well and I don't mind the ads. But I think that it should have an option to play in shuffle or play in the order the songs are in; I make my playlists in the order I like them, so shuffle being the only option makes it frustrating."
                Topics / Keyphrases: ['music streaming experience', 'Spotify app features', 'song customization', 'user frustration', 'playlist creation', 'song previews', '10k song limit', 'shuffle mode', 'period pad ads', 'Spotify Premium recommendation']
                Label: 3

                15. Comment: "I've loved the app so far, the interface... organization..but I was logged out this evening and it's refusing to log me in. This is sad. Please fix it"
                Topics / Keyphrases: ['Spotify app login issues', 'user experience problems', 'email password recovery', 'local files playback', 'account compatibility', 'M-Pesa payment issue', 'phone technical difficulties', 'internet access settings', 'data system recognition', 'app functionality fix']
                Label: 3

                16. Comment: "Great app, great music, love the weekly custom play lists but only negative thing is the pop aps if it not a primum user"
                Topics / Keyphrases: ['Spotify App', 'user comments', 'music streaming', 'positive opinion', 'flaw', 'song downloading', 'user experience', 'song selection', 'Google Play Store', 'app analysis']
                Label: 4

                17. Comment: "I love Spotify but it gets annoying when I try to add an album to a playlist, I have to go through every song and add them by hand. Other than that I find it wonderful 😊"
                Topics / Keyphrases: ['user experience', 'Spotify app', 'playlist customization', 'song selection', 'playback issues', 'premium membership', 'song order', 'music recommendations', 'technical problems', 'app functionality']
                Label: 4

                18. Comment: "I may not remember the song's name so it would be good to have a function where we can just say some key words and the artist. But is amazing."
                Topics / Keyphrases: ['Spotify app user experience', 'song identification and playback issues', 'offline listening limitations', 'song lyrics and information', 'playlist creation and customization', 'music preference and recommendations', 'app navigation and control', 'free version limitations', 'artist compensation and payment', 'user feedback and suggestions']
                Label: 4

                19. Comment: "Love it! Great among quality, perfect sound for songs. My one problem is when it adds songs to your playlists. Other than that its perfect."
                Topics / Keyphrases: ['Spotify app user experience', 'high quality music', 'great sound', 'songs and playlists', 'album cover concern', 'user interface issues', 'app evaluation', 'music variety', 'free version limitations', 'app improvement feedback']
                Label: 4

                20. Comment: "This is a good music app overall but the only thing anoying is there's so much ads that say thank you for listening to Spotify it's so anoying it's like made.ti anoy people but overall best music app ever"
                Topics / Keyphrases: ['Spotify App', 'music service', 'podcasts', '5 stars', 'user experience', 'music discovery', 'Joe Rogan Podcast', 'Google Play Store', 'fast forward', 'self-remove artists']
                Label: 4

                21. Comment: "SPOTIFY IS MY DAMN JAM!!! I'D PICK IT OVER ANY OTHER MUSIC APP OUT THERE ESPECIALLY NOW THAT THEY HAVE ADDED THE LYRICS TO SONGS!!! My only request is to have the option to keep ur smartphone screen awake if u have the lyrics pulled up!! Love love love you Spotify Folk!!!!"
                Topics / Keyphrases: ['Spotify app issues', 'trending albums refresh', 'annoying bugs', 'saving albums/playlists', 'user experience', 'music playback problems', 'app improvement requests', 'playlist limitations', 'hidden favorite tracks', 'app customization options']
                Label: 5

                22. Comment: "The app is great i regualry listen to lots of fantastic music and premium is very good because there are no adds! So i definetley reccomend you install it!:):):):)"
                Topics / Keyphrases: ['Spotify App', 'user experience', 'music streaming', 'playlist', 'song selection', 'premium feature', 'app performance', 'user feedback', 'WiFi connection issue', 'AI and 
                ads']
                Label: 5

                23. Comment: "I have this app for 8 years and counting and I cannot live without it! Spotify accompanies me everywhere I go. Thank you!"
                Topics / Keyphrases: ['best music streaming app', 'personalized music playlists', 'user experience', 'Spotify app usage', 'music preference', 'positive user feedback', 'app loyalty', 'Google Play Store', 'daily music listening', 'music app enhancement']
                Label: 5

                24. Comment: "so far this is the best music app I have used rarely crash's Plays offline has a multitude of features and functions and very easy to manuver around. Thanks so much guy's and gal's for all your hard work"
                Topics / Keyphrases: ['Spotify app usability', 'music customization and recommendations', 'app continuity and features', 'phone usage and music streaming', 'user-friendly interactive experience', 'satisfied long-term user experience', 'convenient music listening and accessibility', 'easy-to-navigate international platform', 'music discovery and song education', 'cost-effective artist platform']
                Label: 5

                25. Comment: "I just Love Spotify. It has every single song/music that is, was, and will be a part of my life's music repritoire. Thanks for supplying such a great array of listening pleasures, SPOTIFY!!"
                Topics / Keyphrases: ['Spotify as top music streaming platform', 'growing music and podcast options', 'supporting underground artists', 'Spotify for Artists', 'user preferences in audio entertainment', 'headphones usage', 'user experience', 'Spotify app features', 'user's love for Spotify Premium', 'playlist creation and song recommendations']
                Label: 5


                Please provide the sentiment classification as a single number (1, 2, 3, 4, or 5) and nothing more for the following comment:


                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
# list.pop()

In [ ]:
list

In [ ]:
train_data['predictedSentiment'] = pd.Series(list)

In [ ]:
train_data

In [ ]:
import re

def contains_number(s):
    return any(char.isdigit() for char in s)

def findNumbers(string):
    # Find all numbers in the string
    numbers = None
    if(contains_number(string)):
        numbers = re.findall(r'\d+', string)
    else:
        numbers = ""
    
    # Convert the numbers to integers
    
    numbers = [int(x) for x in numbers]
    
    if(len(numbers)>0):
        x = numbers[0]-1
    else:
        x = -1
    return x

In [ ]:
formated = train_data['predictedSentiment'].apply(findNumbers)

In [ ]:
train_data['predictedSentiment'] = formated

In [ ]:
train_data

In [ ]:
train_data.to_csv('mixtralOutput/topics/filenameSpotifyFewShotMixtral-T.csv', index=False)

In [ ]:
results = pd.read_csv("mixtralOutput/filenameSpotifyFewShotMixtral.csv")

In [ ]:
results

In [ ]:
results['predictedSentiment'].value_counts().sort_index()

In [ ]:
filtered_df = results[results["predictedSentiment"] == -1]

In [ ]:
filtered_df

In [ ]:
newResults = results[results["predictedSentiment"] != -1]

In [ ]:
newResults

In [ ]:
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, classes, ymap=None, figsize=(17,17)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      classes:   aliases for the labels. String array to be shown in the cm plot.
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    sns.set(font_scale=2.8)

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            #elif c == 0:
            #    annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize='true')
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm = cm * 100
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    plt.yticks(va='center')

    sns.heatmap(cm, annot=annot, fmt='', ax=ax, xticklabels=classes, cbar=True, cbar_kws={'format':PercentFormatter()}, yticklabels=classes, cmap="Blues")
    plt.savefig(f'{filename}.png',  bbox_inches='tight')
    plt.savefig(f'{filename}.svg',  bbox_inches='tight')

In [ ]:
# Generate the confusion matrix
unique_labels = np.sort(newResults['scoreSA'].unique())
mapping = {0: 'Very Negative (1)', 1: 'Negative (2)', 2: 'Neutral (3)', 3: 'Positive (4)', 4: 'Very Positive (5)'}

# Applying mapping
classes = [mapping[value] for value in unique_labels]

In [ ]:
cm_analysis(newResults['scoreSA'], newResults['predictedSentiment'], 'API', unique_labels, classes)

In [ ]:
accuracy

In [ ]:
balanced_accuracy = balanced_accuracy_score(results['scoreSA'], results['predictedSentiment'])

In [ ]:
# Calculate accuracy for each class
for i in range(len(unique_labels)):
    temp = newResults[newResults["scoreSA"] == i]
    value = accuracy_score(temp['scoreSA'], temp['predictedSentiment'])
    print(f'Label:\t{i}\t-\tValue:\t{value}')

In [ ]:
balanced_accuracy

In [ ]:
precision_ALL

In [ ]:
recall_ALL

In [ ]:
f1_ALL

In [ ]:
mae_ALL

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1

In [ ]:
support